# Wrangle MBTA Days with Games #

## Setup ##

In [1]:
import sys
import datetime
import numpy as np
import pandas as pd

## Import Data ##

In [2]:
original = pd.read_csv("../../../data/mbta_daily_weather_noholidays.csv", low_memory=False)
original.head()

,locationid,service_day,entries,name,line_1,line_2,lat,lon,service_datetime,fog,...,rain_predict,rain_fall_predict,snow_predict,snow_fall_predict,snow_accum,snow_accum_predict,dist_to_center,entries_weeks_ago_1,entries_weeks_ago_2,entries_weeks_ago_3
0,1002,2013-01-03 00:00:00,5733,Andrew Square,Red,NaN,42.32955,-71.05696,2013-01-03 05:00:00,0,...,0,0,0,0,0,0,3.404767,NaN,NaN,NaN
1,1002,2013-01-04 00:00:00,6125,Andrew Square,Red,NaN,42.32955,-71.05696,2013-01-04 05:00:00,0,...,0,0,0,0,0,0,3.404767,NaN,NaN,NaN
2,1002,2013-01-05 00:00:00,3410,Andrew Square,Red,NaN,42.32955,-71.05696,2013-01-05 04:15:00,0,...,0,0,1,0,0,0,3.404767,NaN,NaN,NaN
3,1002,2013-01-06 00:00:00,2311,Andrew Square,Red,NaN,42.32955,-71.05696,2013-01-06 05:00:00,0,...,0,0,0,0,0,0,3.404767,NaN,NaN,NaN
4,1002,2013-01-07 00:00:00,5998,Andrew Square,Red,NaN,42.32955,-71.05696,2013-01-07 04:00:00,0,...,0,0,0,0,0,0,3.404767,NaN,NaN,NaN


In [4]:
games = pd.read_csv("../../../data/game_days.csv", low_memory=False)
print(games.columns)
games.head()

Index([u'date', u'weekday', u'bruins_playoff', u'bruins_opponent', u'bruins_time', u'celtics_playoff', u'celtics_opponent', u'celtics_time', u'sox_time', u'sox_opponent', u'sox_time_rounded', u'bruins_game', u'celtics_game', u'sox_game', u'bruins_mon', u'bruins_tue', u'bruins_wed', u'bruins_thu', u'bruins_fri', u'bruins_sat', u'bruins_sun', u'bruins_mon_thu', u'bruins_mon_fri', u'bruins_fri_sat', u'celtics_mon', u'celtics_tue', u'celtics_wed', u'celtics_thu', u'celtics_fri', u'celtics_sat', u'celtics_sun', u'celtics_mon_thu', u'celtics_mon_fri', u'celtics_fri_sat', u'sox_mon', u'sox_tue', u'sox_wed', u'sox_thu', u'sox_fri', u'sox_sat', u'sox_sun', u'sox_mon_thu', u'sox_mon_fri', u'sox_fri_sat', u'bruins_time_early', u'bruins_time_late', u'celtics_time_early', u'celtics_time_late', u'sox_time_early', u'sox_time_late', u'bruins_mon_early', u'bruins_mon_late', u'bruins_tue_early', u'bruins_tue_late', u'bruins_wed_early', u'bruins_wed_late', u'bruins_thu_early', u'bruins_thu_late', u'bruin

,date,weekday,bruins_playoff,bruins_opponent,bruins_time,celtics_playoff,celtics_opponent,celtics_time,sox_time,sox_opponent,...,sox_sat_early,sox_sat_late,sox_sun_early,sox_sun_late,sox_mon_thu_early,sox_mon_thu_late,sox_mon_fri_early,sox_mon_fri_late,sox_fri_sat_early,sox_fri_sat_late
0,2012-04-13,4,NaN,NaN,NaN,NaN,NaN,NaN,14:05:00,Rays,...,0,0,0,0,0,0,1,0,1,0
1,2012-04-14,5,NaN,NaN,NaN,NaN,NaN,NaN,16:05:00,Rays,...,1,0,0,0,0,0,0,0,1,0
2,2012-04-15,6,NaN,NaN,NaN,NaN,NaN,NaN,13:35:00,Rays,...,0,0,1,0,0,0,0,0,0,0
3,2012-04-16,0,NaN,NaN,NaN,NaN,NaN,NaN,11:05:00,Rays,...,0,0,0,0,1,0,1,0,0,0
4,2012-04-17,1,NaN,NaN,NaN,NaN,NaN,NaN,19:10:00,Rangers,...,0,0,0,0,0,1,0,1,0,0


In [23]:
subgames = games[['date', 'bruins_game', 'celtics_game', 'sox_game']].copy()

original['service_day'] = pd.to_datetime(original['service_day'])
subgames['date'] = pd.to_datetime(subgames['date'])

out = pd.merge(original, subgames, how='left', left_on='service_day', right_on='date')

out.drop('date', axis=1, inplace=True)
out['bruins_game'] = out['bruins_game'].fillna(0)
out['celtics_game'] = out['celtics_game'].fillna(0)
out['sox_game'] = out['sox_game'].fillna(0)


print(len(original))
print(len(out))

#print(subgames.head())
print(len(subgames['date']))
print(len(subgames['date'].unique()))

out.head()

45968
45968
465
465


,locationid,service_day,entries,name,line_1,line_2,lat,lon,service_datetime,fog,...,snow_fall_predict,snow_accum,snow_accum_predict,dist_to_center,entries_weeks_ago_1,entries_weeks_ago_2,entries_weeks_ago_3,bruins_game,celtics_game,sox_game
0,1002,2013-01-03,5733,Andrew Square,Red,NaN,42.329550,-71.056960,2013-01-03 05:00:00,0,...,0,0,0,3.404767,NaN,NaN,NaN,0,0,0
1,1004,2013-01-03,5842,JFK/U Mass,Red,NaN,42.321438,-71.052393,2013-01-03 04:45:00,0,...,0,0,0,4.328881,NaN,NaN,NaN,0,0,0
2,1005,2013-01-03,6139,North Quincy,Red,NaN,42.274816,-71.029176,2013-01-03 03:15:00,0,...,0,0,0,9.777437,NaN,NaN,NaN,0,0,0
3,1006,2013-01-03,4028,Wollaston,Red,NaN,42.265615,-71.019402,2013-01-03 05:00:00,0,...,0,0,0,10.976943,NaN,NaN,NaN,0,0,0
4,1007,2013-01-03,7331,Quincy Center,Red,NaN,42.250879,-71.004798,2013-01-03 04:30:00,0,...,0,0,0,12.909591,NaN,NaN,NaN,0,0,0


In [24]:
out.to_csv("../../../data/mbta_daily_weather_games_noholidays.csv", index=False)